In [38]:
!pip3 install flair --quiet

You should consider upgrading via the '/home/rkozik/anaconda3/bin/python -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import RepeatedStratifiedKFold

data = pd.read_csv("raw/covid_fake_news.csv",sep="\t")
data["label"] = 1- data["label"]
print("labels\n",data["label"].value_counts())
print("shape  \n",data.shape)

bootstrap_size = 0

if bootstrap_size != 0:
    bootstrap_factor = bootstrap_size / data.shape[0]
    bootstrap = np.random.uniform(size=data.shape[0]) < bootstrap_factor
    data = data.iloc[bootstrap]

X, y = data["title"].values, data["label"].values


rskf = RepeatedStratifiedKFold(n_splits=2, n_repeats=5, random_state=1410)
foldids = []
for fold_idx, (train, test) in tqdm(enumerate(rskf.split(X, y)), total=rskf.get_n_splits()):
    foldids.append((fold_idx,train,test))
    
print("shapes X",X.shape,"y", y.shape)

labels
 1    8511
0     461
Name: label, dtype: int64
shape  
 (8972, 2)


100%|█████████████████████████████████████████| 10/10 [00:00<00:00, 2304.94it/s]

shapes X (8972,) y (8972,)


In [67]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import RepeatedStratifiedKFold

data = pd.read_csv("raw/pubhealth.csv",sep=",")
print("labels\n",data["label"].value_counts())
print("shape  \n",data.shape)

bootstrap_size = 0

if bootstrap_size != 0:
    bootstrap_factor = bootstrap_size / data.shape[0]
    bootstrap = np.random.uniform(size=data.shape[0]) < bootstrap_factor
    data = data.iloc[bootstrap]

X, y = data["text"].values, data["label"].values

rskf = RepeatedStratifiedKFold(n_splits=2, n_repeats=5, random_state=1410)
foldids = []
for fold_idx, (train, test) in tqdm(enumerate(rskf.split(X, y)), total=rskf.get_n_splits()):
    foldids.append((fold_idx,train,test))
    
print("shapes X",X.shape,"y", y.shape)

labels
 1    6306
0    3769
Name: label, dtype: int64
shape  
 (10075, 11)


100%|█████████████████████████████████████████| 10/10 [00:00<00:00, 1794.12it/s]

shapes X (10075,) y (10075,)


In [2]:
from flair.models import TARSClassifier
from flair.data import Sentence

import flair, torch
flair.device = torch.device('cuda')

# 1. Load our pre-trained TARS model for English
tars = TARSClassifier.load('tars-base')


2022-09-24 08:39:34,934 loading file /home/rkozik/.flair/models/tars-base-v8.pt


In [33]:

# 2. Prepare a test sentence
sentence = Sentence("the student fell ill with covid")

# 3. Define some classes that you want to predict using descriptive names
classes = ["Health, Medicine",
           "Entertainment, Art, Culture", 
           "Computers, Science, Technology", 
           "Finance","Economics",
           "Lifestyle",
           "Sport, Leisure",
           "Politics",
           "Opinion, Recommendation, Review",
           "Traveling"
          ]

#4. Predict for these classes
tars.predict_zero_shot(sentence, classes)

# Print sentence with predicted labels
print(sentence)

Sentence: "the student fell ill with covid"   [− Tokens: 6  − Sentence-Labels: {'Health, Medicine-Entertainment, Art, Culture-Computers, Science, Technology-Finance-Economics-Lifestyle-Sport, Leisure-Politics-Opinion, Recommendation, Review-Traveling': [Health, Medicine (0.7386)]}]


In [4]:

sentence = Sentence(X[8])
tars.predict_zero_shot(sentence, classes)

import numpy as np
def get_best_lab(sentence):
    labels = sentence.to_dict()["labels"]
    if len(labels) > 0:
        return labels[np.argmax([x["confidence"] for x in labels])]["value"]
    else:
        return "none"

print(sentence,get_best_lab(sentence))


Sentence: "Social media posts criticize a photo from smiling president of the Goverment Pedro SÃ ¡ nchez during his holidays after thousands died from COVID-19"   [− Tokens: 24  − Sentence-Labels: {'Health, Medicine-Entertainment, Art, Culture-Computers, Science, Technology-Finance, Economics-Lifestyle-Sport, Leisure-Politics-Opinion, Recommendation, Review-Traveling': [Politics (0.8274)]}] Politics


In [68]:
classes = ["Health", "Medicine", "vaccination cure","virus",
           "Entertainment, Art, Culture", 
           "Computers, Science, Technology", 
           "Finance","Economics","Economy",
           "Lifestyle",
           "Sport, Leisure",
           "Politics",
           "Opinion, Recommendation, Review",
           "Traveling","pandemic","Government"
          ]

#classes = ["happy", "sad", "neutral"]



tt='Photo shows Muslims in Tamil Nadu state of India are floating lockdown rules during Ramzan to offer night prayer'
sentence = Sentence(tt)
tars.predict_zero_shot(sentence, classes)
print(sentence)

Sentence: "Photo shows Muslims in Tamil Nadu state of India are floating lockdown rules during Ramzan to offer night prayer"   [− Tokens: 19]


In [69]:
from tqdm import tqdm
from flair.datasets import DataLoader 

categories = []
sX = [Sentence(x) for x in tqdm(X)]

dataloader = DataLoader(sX, batch_size=32, num_workers=8)
for batch in tqdm(dataloader):
    tars.predict_zero_shot(batch, classes)

for s in tqdm(sX):
    c = get_best_lab(s)
    categories.append(c)
#    c = get_best_lab(s)
#    categories.append(c)

100%|███████████████████████████████████| 10075/10075 [00:03<00:00, 2549.49it/s]


In [70]:
import pickle
# SAVE
with open('pubhealth_categories.pickle', 'wb') as handle:
    pickle.dump(categories, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [71]:
oo=np.unique(categories, return_counts=True)
list(zip(oo[0],oo[1]))

[('Computers, Science, Technology', 136),
 ('Economics', 27),
 ('Economy', 9),
 ('Entertainment, Art, Culture', 213),
 ('Finance', 1),
 ('Government', 804),
 ('Health', 1347),
 ('Medicine', 529),
 ('Opinion, Recommendation, Review', 64),
 ('Politics', 5717),
 ('Sport, Leisure', 72),
 ('Traveling', 12),
 ('none', 1042),
 ('pandemic', 41),
 ('vaccination cure', 22),
 ('virus', 39)]